# Metodo de Numerov para Equações diferenciais de segunda ordem incompletas.

In [22]:
import numpy as np

In [23]:

x = np.linspace(-10,10,1001)
def v(x):
    L = 0
    Vx = -2./x
    return Vx

## Definindo o limite máximo a esquerda

In [24]:
def vec_max(dim, x):
    xmax = 0
    N = dim
    for j in range(int(N)):
        if j<int(N) and abs(x[j])>xmax:
            xmax = abs(x[j])
        else:
            continue
        
        return xmax
    

In [25]:
def nrovl(y0,y1,x0,E,h,iflag):
    q0 = (E-v(x0))
    q1 = (E-v(x0+h))
    q2 = (E-v(x0+2*h))
    p0 = (1+h*h*q0/12)
    p1 = 2*(1-5*h*h*2*q1/12)
    p2 = 1+h*h*q2/12
    y2 = (p1*y1-p0*y0)/p2
    if iflag < 1:
        print(f"x0 = {x0}, y0 = {y0}, V = {v(x0)}")
        print(f"x1 = {x0+h}, y1 = {y1}, V = {v(x0+h)}")
    return y2

In [26]:
def nrovr(y0, y1, x0, E,h, iflag):
    q0 = (E-v(x0))
    q1 = (E-v(x0-h))
    q2 = (E-v(x0-2*h))
    p0 = (1.0+ h**2*q0/12.0)
    p1 = 2*(1-5*h**2*q1/12.0)
    p2 = (1.0+h**2*q2/12.0)
    y2 = (p1*y1-p0*y0)/p2
    if iflag <1:
        print(f"x_100 = {x0} y0_100 = {y0} V = {V(x0)}")
        print(f"x_99 = {x0-h} y0_99 = {y1} V = {V(x0-h)}")
    return y2

In [27]:
def espectro(xl, xu,h,delta, eps,dim,nmax,kmax, Ein, vmax, dE, iflag ):
    xx = list(range(dim))
    yy = list(range(dim))
    ww= list(range(dim))
    yl = list(range(dim))
    yr = list(range(dim))
    ee = list(range(nmax))
    ff = list(range(nmax))
    ff2 = list(range(nmax))
    yy1 = list(range(dim))
    yy2 = list(range(dim))
    yy3 = list(range(dim))
    cores = ['b','r','g','m','c']
    nk = list(range(nmax))
    E_old = Ein
    E = Ein + dE
    
    for M in range(nmax):
        print(f"{5*'*'} Eigenvalue {M+1} {5*'*'}")
        f_old = 0
        for k in range(kmax):
            imatch = 0
    for j in range(dim-1):
        xx[0] = xl
        xx[dim-1]  = xu
        DE1 = E - v(xx[j])
        xx[j+1] = xx[j] + h
        DE2 = E - v(xx[j+1])
        D1D2 = DE1*DE2
        if D1D2 <= 0 and DE1>0:
            imatch = j+1
            print(f"imatch = {imatch} xmatch = {xx[imatch]} V = {v(xx[imatch])} E = {E}")
            
    xmatch = xx[imatch]
    ii=range(imatch+2)
    i_lim = ii[2:imatch+2]
    xx[0]= xl
    xx[1]= xl + h
    yy[0] = yl[0]
    yy[1] = delta
    for i in i_lim:
        yy[i] = nrovl(yy[i-2], yy[i-1], xx[i-2], E, h, iflag)
        xx[i] = xx[i-1] + h
        
    jjj = list(range(dim+1))
    j_lim = list(jjj[imatch+1:dim+1])
    comp_j = len(j_lim)
    jj = sorted(j_lim,key=abs, reverse=True)
    
    for i in range(dim):
        if i <= imatch+1:
            yl[i] = yy[i]
        if i > imatch+1:
            yl[i] = 0   
    for i in jj:
        if i ==dim:
            yr[dim-1] = 0
        if i == dim-1:
            yr[dim-2] = 2*delta
        if i < dim-1:
            yr[i] = nrovr(yr[i-2], yr[i-1], xx[i-2], E, h, iflag)
            xx[i] = xx[i+1] - h
            
            
    for i in range(imatch -1):
        yr[i] = 0
        
        ymatch = yy[imatch]
        yrmatch = yr[imatch]
        ylmatch = yl[imatch]
        
        if ymatch != 0:
            scale = yrmatch/ymatch
        else:
            continue
        
    for t in range(imatch+1):
        yy[t] = yy[t]*scale
        yl[t] = -yl[t]*scale
    
    yl[imatch+1] = -yl[imatch+1]*scale
    ymatch = yy[imatch]
    dlmatch = yy[imatch+1]*scale - yy[imatch-1]
    
    t_lim = list(range(dim+1))
    tt = list(t_lim[imatch+1:dim])
    
    drmatch = yr[imatch+1] - yr[imatch-1]
    
    f = (dlmatch - drmatch)/(2*h)
    
    for t in tt:
        yy[t] = yr[t]
        
        delta_E = -f*(E-E_old)/(f-f_old)
        
        if abs(delta_E)<eps:
            ee[M] = E
            nk[M] = k
            ff[M] = f
            k = kmax
            break
    
    else:
        f_old = f
        E_old = E
        E = E + 24*dE 
    
    if M == 0:
        E = E +126*dE
        for j in range(dim):
            yy1[j] = yy[j]
    if M==1:
        E = E + 3*dE
        for j in range(dim):
            yy2[j]=yy[j]
    if M==2:
        E = E_old + 6*dE
        for j in range(dim):
            yy3[j] = yy[j]
            
        print(f"k = {nk[M]} E = {ee[M]} E_old = {E_old} f_old = {f_old} f = {f} delta_E = {delta_E}\n\n")
    
    
    return ee,xx,yy1,yy2,yy3
    
    

In [28]:
a = 0.001; b = 60.001; h = 0.01; xl =a;xu=b; D= xu-xl
delta = 0.02;eps = 0.0001;dim=int(D/h); kmax = 300;nmax = 3
n=0;iflag =0
Rydberg=13.605693122994
x0 = xl; y0=0;y1=delta;iflag=1

nrovl(y0,y1,x0,0,h,iflag)
nrovr(y0,y1,x0,0,h,iflag)
dE = delta/4
Ein = -1.6
vmax = 0.
print(20*'=')
print()
print(f"Potencial: V(x) = -2/x (Hidrogênio L=0)")
print()
print(20*'=')

print(f"Ein: {Ein:.3f} dE = {dE:.3f} h = {h:.3f} dim = {dim}\n")
colors = ['r','g','b','m','c']
ee,xx,yy1,yy2,yy3 = espectro(xl,xu,h,delta,eps,dim,nmax, kmax, Ein, dE,vmax, iflag)
A = 1
ymax1 = vec_max(dim, yy1)
ymax2 = vec_max(dim, yy2)
ymax3 = vec_max(dim, yy3)

for i in range(dim):
    yy1[i] = A*yy1[i]/ymax1
    yy2[i] = A*yy2[i]/ymax2
    yy3[i] = A*yy3[i]/ymax3


Potencial: V(x) = -2/x (Hidrogênio L=0)

Ein: -1.600 dE = 0.005 h = 0.010 dim = 6000

***** Eigenvalue 1 *****
***** Eigenvalue 2 *****
***** Eigenvalue 3 *****
imatch = 125 xmatch = 1.2510000000000008 V = -1.598721023181454 E = -1.6
k = 299 E = -1.6 E_old = -1.6 f_old = 0 f = -366.8772790558954 delta_E = -0.0


